<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=33f0aab22c16a5bdbe95c33ea0e11e7dc810f0912fa7921a67e2e4bc92aefdee
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-26 11:22:16
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -29.20 K (-0.21%)
Current PnL: -25.64 L (-16.56%)
CY Booked + Current PnL: -11.35 L (-7.33%)
-------------------
Total profit:  1.56 L
Total loss:  -27.20 L
-------------------
Total Booked + Current PnL: 15.63 L (12.23%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.11%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.27 L (63.84%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.69,-18.68,23.08,0.09,18915.0,-18823.0,81954.0,90.18,42.0,M-SC,2.84,253.0,-1.00,0.58,7.67,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.18,9.30,9.50,19.69,19910.0,17838.0,209580.0,-1.35,57.0,X-MC,2.46,78.0,0.90,1.49,16.73,XY25,NTT,AC
50,MASFIN,398.61,-1.48,-3.12,25.98,22.05,24660.0,-3060.0,94920.0,-16.44,51.0,H-SC,6.82,164.0,-0.12,0.67,37.97,XR,ATH,FINANCE
52,NESTLEIND,1377.00,0.40,11.50,8.88,21.40,26279.0,30514.0,295940.0,3.40,59.0,X-LC,5.60,12.0,1.16,2.10,18.90,XY25,NTT,FMCG
43,ITC,452.00,-0.48,-0.82,11.70,10.78,27886.0,-1979.0,238339.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.37,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,4.51,-15.91,73.32,45.74,149307.0,-38539.0,203638.0,12.11,69.0,H-SC,6.21,174.0,-0.26,1.45,32.45,XY24,NTT,ENGINEERING
42,IRCTC,1269.0,3.51,-15.22,80.38,52.93,184344.0,-41167.0,229341.0,-37.32,64.0,H-MC,6.95,118.0,-0.22,1.63,6.38,XY24,BTT,TRAVEL
60,REPCOHOME,880.0,3.02,-11.30,114.01,89.82,289724.0,-32383.0,254122.0,-56.04,56.0,H-SC,5.52,160.0,-0.11,1.81,34.45,XY24,NTT,FINANCE
66,SIS,528.0,2.56,-21.68,54.84,21.26,47686.0,-24077.0,86955.0,2054.92,62.0,H-SC,4.84,166.0,-0.50,0.62,17.48,OX40N,NTT,MISC
44,JCHAC,2282.0,2.36,-35.76,55.66,-0.01,45695.0,-45709.0,82096.0,16704.35,38.0,M-SC,12.09,233.0,-1.00,0.58,5.07,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
47,KANSAINER,340.00,-2.48,-21.16,43.93,13.47,93394.0,-57069.0,212598.0,-67.53,56.0,H-SC,6.81,158.0,-0.61,1.51,8.10,XY24,NTT,PAINTS
37,IEX,219.00,-2.22,-7.67,60.97,48.63,112563.0,-15334.0,184620.0,-38.61,38.0,H-SC,13.42,136.0,-0.14,1.31,3.00,XR,NTT,MISC
69,SURYODAY,216.00,-1.58,-21.22,56.17,23.03,79242.0,-37995.0,141076.0,53.51,50.0,H-SC,6.80,167.0,-0.48,1.00,39.98,XR,NTT,BANKS
22,DIXON,18931.72,-1.56,-17.21,52.28,26.07,84493.0,-33605.0,161616.0,-59.72,29.0,X-MC,13.17,56.0,-0.40,1.15,0.66,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,0.41,-0.65,20.01,19.23,29911.0,-980.0,149480.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.49,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.19,-2.23,112.65,107.91,162616.0,-3289.0,144355.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.03,7.63,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.68,-18.59,22.86,0.02,32485.0,-32455.0,142104.0,116.52,36.0,M-SC,10.61,234.0,-1.00,1.01,23.61,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.69,-18.68,23.08,0.09,18915.0,-18823.0,81954.0,90.18,42.0,M-SC,2.84,253.0,-1.00,0.58,7.67,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.48,-21.16,43.93,13.47,93394.0,-57069.0,212598.0,-67.53,56.0,H-SC,6.81,158.0,-0.61,1.51,8.10,XY24,NTT,PAINTS
66,SIS,528.00,2.56,-21.68,54.84,21.26,47686.0,-24077.0,86955.0,2054.92,62.0,H-SC,4.84,166.0,-0.50,0.62,17.48,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-1.28,1.95,78.20,81.67,74790.0,1829.0,95639.0,8616.67,74.0,L-SC,14.19,271.0,0.02,0.68,85.21,XR,NTT,CERAMICS
84,WIPRO,420.0,-0.35,9.56,57.23,72.27,106246.0,16200.0,185648.0,-6.08,65.0,M-LC,4.16,99.0,0.15,1.32,16.33,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.35,9.56,57.23,72.27,106246.0,16200.0,185648.0,-6.08,65.0,M-LC,4.16,99.0,0.15,1.32,16.33,XR,NTT,IT
6,ASIANTILES,137.00,-1.28,1.95,78.20,81.67,74790.0,1829.0,95639.0,8616.67,74.0,L-SC,14.19,271.0,0.02,0.68,85.21,XR,NTT,CERAMICS
25,FINCABLES,1641.55,-0.19,-2.23,112.65,107.91,162616.0,-3289.0,144355.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.03,7.63,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.02,-4.12,115.61,106.72,136713.0,-5082.0,118254.0,-52.54,41.0,H-SC,4.75,139.0,-0.04,0.84,18.29,AR,ATH,MISC
1,ABB,7934.00,0.08,-4.25,52.02,45.57,130316.0,-11107.0,250512.0,-38.46,54.0,H-MC,4.52,120.0,-0.09,1.78,6.60,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.56,-17.21,52.28,26.07,84493.0,-33605.0,161616.0,-59.72,29.0,X-MC,13.17,56.0,-0.40,1.15,0.66,X40N,ATH,IT
8,AWL,485.00,-0.32,-25.25,105.33,53.50,237594.0,-76180.0,225571.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.19,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.13,-18.38,90.49,55.47,179681.0,-44722.0,198564.0,-27.60,30.0,X-MC,16.06,64.0,-0.25,1.41,1.74,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.26,-18.60,76.07,43.32,100854.0,-30292.0,132580.0,-30.03,30.0,X-SC,6.28,90.0,-0.30,0.94,1.80,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.34,-27.15,125.32,64.15,217242.0,-64601.0,173350.0,-57.19,32.0,X-SC,4.53,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-0.82,-1.91,23.50,21.14,52403.0,-4346.0,222993.0,-8.88,51.0,X-MC,1.15,74.0,-0.08,1.58,25.69,XY24,NTT,PAINTS
15,CAMS,950.0,-0.84,-1.24,26.04,24.48,64767.0,-3135.0,248721.0,-81.09,46.0,X-SC,1.64,86.0,-0.05,1.77,21.61,X40N,NTT,MISC
78,UNITDSPR,1644.0,0.43,4.91,15.11,20.77,37118.0,11506.0,245650.0,-4.29,51.0,X-MC,1.88,62.0,0.31,1.75,12.17,X40N,NTT,BREWERIES
43,ITC,452.0,-0.48,-0.82,11.70,10.78,27886.0,-1979.0,238339.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.37,X40,NTT,FMCG
20,DABUR,735.0,-0.53,-4.09,49.83,43.70,118554.0,-10151.0,237917.0,-11.99,33.0,X-MC,2.11,72.0,-0.09,1.69,10.34,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.34,-27.15,125.32,64.15,217242.0,-64601.0,173350.0,-57.19,32.0,X-SC,4.53,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.32,-25.25,105.33,53.50,237594.0,-76180.0,225571.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.19,XY24,NTT,FMCG
22,DIXON,18931.72,-1.56,-17.21,52.28,26.07,84493.0,-33605.0,161616.0,-59.72,29.0,X-MC,13.17,56.0,-0.40,1.15,0.66,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.15,-38.83,121.05,35.20,95268.0,-49969.0,78701.0,3.25,35.0,X-SC,17.16,92.0,-0.52,0.56,0.95,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,0.82,-19.86,76.59,41.51,161643.0,-52315.0,211050.0,-4.54,42.0,X-MC,7.98,61.0,-0.32,1.50,1.11,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.97,-27.48,96.98,42.84,45716.0,-17866.0,47140.0,-52.62,53.0,X-SC,33.31,83.0,-0.39,0.33,6.65,XY24,NTT,MISC
59,RELAXO,1176.00,-0.58,-47.81,191.81,52.31,145323.0,-69396.0,75764.0,-44.44,43.0,X-SC,7.21,91.0,-0.48,0.54,1.41,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.15,-38.83,121.05,35.20,95268.0,-49969.0,78701.0,3.25,35.0,X-SC,17.16,92.0,-0.52,0.56,0.95,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.54,-1.07,25.42,24.08,31925.0,-1361.0,125589.0,-9.02,47.0,X-SC,6.65,89.0,-0.04,0.89,18.33,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.26,-18.60,76.07,43.32,100854.0,-30292.0,132580.0,-30.03,30.0,X-SC,6.28,90.0,-0.30,0.94,1.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.95,-9.61,33.54,20.71,114670.0,-36347.0,341890.0,-22.01,63.0,X-LC,3.00,1.0,-0.32,2.43,14.24,X40,ATH,IT
41,INFY,1972.00,-0.32,12.88,18.93,34.24,66856.0,40287.0,353175.0,-12.06,65.0,X-LC,6.46,2.0,0.60,2.51,22.22,X40,NTT,IT
75,TMPV,600.00,-1.06,-27.16,68.82,22.98,113243.0,-61348.0,164550.0,-24.30,46.0,X-LC,6.22,3.0,-0.54,1.17,3.49,XY24,NTT,AUTO
81,VBL,671.64,0.02,-3.49,40.58,35.67,123689.0,-11038.0,304804.0,-14.99,54.0,X-LC,3.33,4.0,-0.09,2.17,9.88,X40N,ATH,FMCG
43,ITC,452.00,-0.48,-0.82,11.70,10.78,27886.0,-1979.0,238339.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.37,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.72,-36.94,110.74,32.89,53916.0,-28520.0,48687.0,-701.24,56.0,L-MC,6.91,259.0,-0.53,0.35,34.08,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,-1.48,-3.12,25.98,22.05,24660.0,-3060.0,94920.0,-16.44,51.0,H-SC,6.82,164.0,-0.12,0.67,37.97,XR,ATH,FINANCE
6,ASIANTILES,137.00,-1.28,1.95,78.20,81.67,74790.0,1829.0,95639.0,8616.67,74.0,L-SC,14.19,271.0,0.02,0.68,85.21,XR,NTT,CERAMICS
13,BSOFT,831.70,0.02,-18.51,85.03,50.79,97463.0,-26031.0,114622.0,4.37,68.0,H-SC,9.26,151.0,-0.27,0.81,33.46,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.28,1.95,78.20,81.67,74790.0,1829.0,95639.0,8616.67,74.0,L-SC,14.19,271.0,0.02,0.68,85.21,XR,NTT,CERAMICS
13,BSOFT,831.70,0.02,-18.51,85.03,50.79,97463.0,-26031.0,114622.0,4.37,68.0,H-SC,9.26,151.0,-0.27,0.81,33.46,XR,ATH,IT
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-1.48,-42.65,203.68,74.15,191099.0,-69782.0,93823.0,-269.06,53.0,H-SC,21.80,149.0,-0.37,0.67,20.27,XR,NTT,CHEMICALS
17,COALINDIA,484.83,0.41,-0.65,20.01,19.23,29911.0,-980.0,149480.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.49,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.88
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.06
MC    30.77
LC    24.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.88
X40      23.03
X40N     12.88
XR        9.86
XY25      9.17
AR        8.85
OX40N     7.60
SR        0.99
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
H-SC    23.92
X-LC    20.71
M-SC    11.53
X-SC     8.04
H-MC     4.84
L-SC     1.57
M-MC     1.40
M-LC     1.32
H-LC     1.10
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.25,-7.45,41.99
IT,13.03,-14.61,72.24
FINANCE,9.67,-17.04,67.95
MISC,7.20,-28.24,80.70
ELECTRICAL,5.95,-11.92,52.92
PAINTS,5.83,-12.49,29.20
INSURANCE,4.69,-2.82,38.01
PHARMA,4.05,-3.05,35.48
AUTO,2.83,-31.71,77.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3138711.0,21
AR,1324961.0,10
XR,1275659.0,13
X40,980709.0,14
X40N,874965.0,9
OX40N,723619.0,10
XY25,350174.0,6
SR,281197.0,2
MH,77186.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3614844.0,25
M-SC,1436635.0,15
X-MC,1422445.0,16
X-LC,811555.0,11
X-SC,786492.0,8
H-MC,397642.0,3
L-SC,240995.0,3
M-LC,106246.0,1
H-LC,78148.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1246990.0      6
           AR         907980.0      5
M-SC       XY24       814470.0      6
H-SC       XR         778751.0      7
X-MC       X40        487979.0      7
           XY24       408551.0      3
           X40N       344362.0      4
H-SC       OX40N      322740.0      4
M-SC       OX40N      319997.0      5
X-LC       X40        312065.0      5
X-SC       X40N       310944.0      3
           XY24       294883.0      3
H-SC       SR         281197.0      2
X-LC       X40N       219659.0      2
H-MC       AR         213298.0      2
X-LC       XY24       189473.0      2
H-MC       XY24       184344.0      1
X-MC       XY25       181553.0      2
X-SC       X40        180665.0      2
M-SC       XR         176633.0      2
L-SC       XR         160113.0      2
M-SC       AR         125535.0      2
M-LC       XR         106246.0      1
X-LC       XY25        90358.0      2
L-SC       OX40N       80882.0      1
H-LC       AR          78148.0      1
H-SC       MH          77186.0      1
L-MC       XR          53916.0      1
M-MC       XY25        48352.0      1
L-LC       XY25        29911.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
